In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [66]:

train_D = pd.read_csv("Davis_train_origin.csv", header = 0)
test_D = pd.read_csv("Davis_test_origin.csv", header = 0)
val_D = pd.read_csv("Davis_val_origin.csv", header = 0)

train_D.drop("Unnamed: 0", axis = 1, inplace = True)
test_D.drop("Unnamed: 0", axis = 1, inplace = True)
val_D.drop("Unnamed: 0", axis = 1, inplace = True)

unique_drug = train_D['Drug_ID'].value_counts().index + test_D['Drug_ID'].value_counts().index + val_D['Drug_ID'].value_counts().index

for i in unique_drug :
    print(i)
    
print(len(unique_drug))

36592664
47234832
40212994
5657004
16947982
36927988
20503770
14704211
10413764
25970421
25212989
11001354
46914109
29490539
16701436
54131391
57857405
21945640
5611919
70818163
36839264
60821081
29933475
25548452
8358214
31604021
62999484
56583743
21992907
17035037
46261704
10114921
23329716
3391774
31149573
20593797
58503486
52529653
11698135
21729518
31558798
58575674
12510889
13366808
33257016
34857203
19942943
10258631
30744538
27242820
44422319
26337608
61949932
50833236
28050539
10662864
24320643
79120497
28070182
32837026
13632704
318127
30218887
37896759
34839764
58369882
28612018
28142750
68


In [83]:
train_D["Y"].value_counts()

0    17294
1      747
Name: Y, dtype: int64

In [3]:
protein2gene = pd.read_csv("convert_table", delimiter="\t")

protein2gene

,UniProtKB Gene Name ID,Gene name,NCBI gene (formerly Entrezgene) ID
0,A0A023T6R1,MAGOHB,Gene::55110
1,A0A023T787,RBM8A,Gene::9939
2,A0A024QY88,NRG1,Gene::3084
3,A0A024QYR8,TM9SF2,Gene::9375
4,A0A024QYT5,SERPINE1,Gene::5054
...,...,...,...
73808,X6RLV5,DDX5,Gene::1655
73809,X6RLX0,ERC1,Gene::23085
73810,X6RLY7,CACNA2D4,Gene::93589
73811,X6RM00,ERC1,Gene::23085


In [85]:
pubchem2drugbank =  pd.read_csv("mbrole2_conversion06232318.tsv", delimiter='\t')

#Select the dataset that you want to make as (entity1, entity2, rel)
# options val_D, test_D, train_D
origin_dti =  train_D[train_D['Drug_ID'].isin(np.array(pubchem2drugbank['Input']))][['Drug_ID', "Target_ID", "Y"]]

def pub_to_db(x) : 
    return pubchem2drugbank[pubchem2drugbank['Input'] == x]['Output'].values[0]

def pro_to_gene(x) :
    
    result = protein2gene[protein2gene['Gene name'] == x]['NCBI gene (formerly Entrezgene) ID'].values
    
    if len(result) >= 1 :
        return result[0]
    else :
        return 1

dti = pd.DataFrame()

dti['Drug'] = origin_dti['Drug_ID'].apply(pub_to_db)
dti['Target'] = origin_dti['Target_ID'].apply(pro_to_gene)

In [86]:
dti['rel'] = origin_dti["Y"]

In [87]:
dti = dti[dti['Target'] != 1]

In [88]:
dti

,Drug,Target,rel
0,DB08568,Gene::22848,0
2,DB08568,Gene::27,0
3,DB08568,Gene::90,0
4,DB08568,Gene::92,0
7,DB08568,Gene::207,0
...,...,...,...
18028,DB04879,Gene::25989,0
18030,DB04879,Gene::7444,0
18031,DB04879,Gene::7465,0
18032,DB04879,Gene::494551,0


In [89]:
import pandas as pd
import numpy as np
import json

with open('entity_drug.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df = df.T

In [90]:
dti

,Drug,Target,rel
0,DB08568,Gene::22848,0
2,DB08568,Gene::27,0
3,DB08568,Gene::90,0
4,DB08568,Gene::92,0
7,DB08568,Gene::207,0
...,...,...,...
18028,DB04879,Gene::25989,0
18030,DB04879,Gene::7444,0
18031,DB04879,Gene::7465,0
18032,DB04879,Gene::494551,0


In [91]:
#a=a.drop("gene_inco", axis = 1)
temp_list = []
for i in dti['Target'].copy() :
    exist = False
    for cn, j in enumerate(df.index) :
        length = len(df.index)
        if i == j :
            exist = True
            val = int(df[0].loc[j])
            temp_list.append(val)
            break
        elif cn == length -1 :
            temp_list.append(np.nan)
            

In [92]:
dti['gene_enco'] = temp_list

In [93]:
dti

,Drug,Target,rel,gene_enco
0,DB08568,Gene::22848,0,9010.0
2,DB08568,Gene::27,0,6200.0
3,DB08568,Gene::90,0,3726.0
4,DB08568,Gene::92,0,7194.0
7,DB08568,Gene::207,0,2232.0
...,...,...,...,...
18028,DB04879,Gene::25989,0,9801.0
18030,DB04879,Gene::7444,0,2877.0
18031,DB04879,Gene::7465,0,10592.0
18032,DB04879,Gene::494551,0,18827.0


In [94]:
import json

# JSON 파일 경로 설정
json_file = 'node2id.json'

# JSON 파일 읽기
with open(json_file) as f:
    data = json.load(f)
    
    
#data   

In [95]:
temp_list2 = [] 
for i in dti["Drug"].copy() :
    exist = False
    count = 0
    for name, number in data.items() :
        length = len(data)
        if i == name :
            exist = True
            temp_list2.append(number)
            break
        elif count >= length -1 :
            temp_list2.append(np.nan)
        count += 1
len(temp_list2)

4697

In [96]:
dti['drug_enco'] = temp_list2
dti = dti.dropna()
dti['gene_enco'] = dti['gene_enco'].astype(int)
dti['drug_enco'] = dti['drug_enco'].astype(int)


/tmp/ipykernel_1406445/972249599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dti['gene_enco'] = dti['gene_enco'].astype(int)
/tmp/ipykernel_1406445/972249599.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dti['drug_enco'] = dti['drug_enco'].astype(int)


In [112]:
# Find the indices of the rows where column 'A' is equal to 0
zero_indices = dti[dti['rel'] == 0].index
one_indices = dti[dti['rel'] == 1].index
# Randomly select 86 indices from the zero_indices
random_indices = np.random.choice(zero_indices, size=172, replace=False)

# Keep only the rows with the selected random indices
df = dti.loc[np.concatenate((random_indices, one_indices))]
df

,Drug,Target,rel,gene_enco,drug_enco
8138,DB01259,Gene::5261,0,15318,206
14726,DB00398,Gene::8550,0,5994,666
7399,DB00619,Gene::494551,0,18827,637
14724,DB00398,Gene::9448,0,3500,666
14662,DB00398,Gene::2046,0,6698,666
...,...,...,...,...,...
15354,DB01268,Gene::8408,1,4283,1018
15355,DB01268,Gene::9706,1,11304,1018
17284,DB05294,Gene::780,1,8437,1185
17339,DB05294,Gene::3932,1,2439,1185


In [113]:
df[['gene_enco', 'drug_enco', 'rel']].to_csv('train.txt', sep=' ', index=False)

# you have just created one of three datasets if you want to make triplets, repeat the code with changing variable above 